# Nuclio - Training function

## Environment

In [1]:
# nuclio: ignore
import nuclio

### Configurations

In [2]:
%%nuclio config

# Trigger
spec.triggers.retrain.kind = "cron"
spec.triggers.retrain.attributes.interval = "1h"

# Base image
spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.triggers.retrain.kind to 'cron'
%nuclio: setting spec.triggers.retrain.attributes.interval to '1h'
%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


### Commands

In [3]:
%%nuclio cmd

############
# installs #
############

# Utils
pip install pyyaml
pip install pyarrow
pip install pandas
pip install joblib

# Igz DB
pip install v3io_frames --upgrade

# Function
pip install scikit-learn==0.20.1
pip install xgboost --upgrade
pip install dask["complete"] --upgrade
pip install dask-ml["complete"] --upgrade

Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Requirement already up-to-date: v3io_frames in /User/.pythonlibs/lib/python3.6/site-packages (0.5.3)
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
  Using cached https://files.pythonhosted.org/packages/6a/49/7e10686647f741bd9c8918b0decdb94135b542fe372ca1100739b8529503/xgboost-0.82-py2.py3-none-manylinux1_x86_64.whl
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Requirement already up-to-date: dask[complete] in /User/.pythonlibs/lib/python3.6/site-packages (1.2.2)
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Requirement already up-to-date:

### Variables

In [52]:
%%nuclio env

# DB Config
V3IO_FRAMESD=${V3IO_FRAMESD}
V3IO_USERNAME=${V3IO_USERNAME}
V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}

# Features
FEATURES_TABLE=netops_features

# Training
TRAIN_ON_LAST=1d
TRAIN_SIZE=0.7

# Parallelizem
NUMBER_OF_SHARDS=4

# Model
MODEL_FILENAME=netops.v3.model
SAVE_TO=/v3io/bigdata/netops/models

%nuclio: setting 'V3IO_FRAMESD' environment variable
%nuclio: setting 'V3IO_USERNAME' environment variable
%nuclio: setting 'V3IO_ACCESS_KEY' environment variable
%nuclio: setting 'FEATURES_TABLE' environment variable
%nuclio: setting 'TRAIN_ON_LAST' environment variable
%nuclio: setting 'TRAIN_SIZE' environment variable
%nuclio: setting 'NUMBER_OF_SHARDS' environment variable
%nuclio: setting 'MODEL_FILENAME' environment variable
%nuclio: setting 'SAVE_TO' environment variable


%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line


## Function

### Imports

In [53]:
# Utils
import os
import time
import yaml
import pandas as pd
import datetime
import itertools
import pickle
import joblib

# DB Connection
import v3io_frames as v3f

# Parallelization
import dask.dataframe as dd
from dask.distributed import Client

# Function
import dask_ml.model_selection as dcv
import xgboost as xgb

### Helper functions

In [54]:
def get_data(context):
    df = context.v3f.read(backend='tsdb', query=f'select * from {context.features_table}',
                          start=f'now-{context.train_on_last}', end='now', multi_index=True)
    df = df.reset_index(drop=True)
    df = df[sorted(df.columns)]
    df = dd.from_pandas(df, npartitions=context.shards)
    return df

In [55]:
def get_train_test_sets_from_data(context, df: pd.DataFrame):
    drop_columns = [col for col in df.columns if 'is_error' in col]
    X = df.drop(drop_columns, axis=1)
    y = df.loc[:, 'is_error']
    X_train, X_test, y_train, y_test = dcv.train_test_split(X, y, train_size=context.train_size, test_size=1-context.train_size)
    return X_train, X_test, y_train, y_test

### Init context

In [56]:
def init_context(context):
    
    # Create our DB client
    v3io_client = v3f.Client(address='http://' + os.getenv('V3IO_FRAMESD', 'framesd:8081'), 
                        container='bigdata', 
                        password=os.environ['V3IO_ACCESS_KEY'], 
                        user=os.environ['V3IO_USERNAME'])
    setattr(context, 'v3f', v3io_client)
    
    # Create Dask client
    dask_client = Client()
    setattr(context, 'dask', dask_client)
    
    # Set time to train on
    train_on_last = os.getenv('TRAIN_ON_LAST', '7d')
    setattr(context, 'train_on_last', train_on_last)
    
    # Set training set size
    train_set_size = float(os.getenv('TRAIN_SIZE', 0.7))
    setattr(context, 'train_size', train_set_size)
    
    # Netops metrics table
    setattr(context, 'features_table', os.getenv('FEATURES_TABLE', 'netops_features'))
    
    # Dask shards / CV
    setattr(context, 'shards', int(os.getenv('NUMBER_OF_SHARDS', 4)))
    
    # Create save-to folder if needed
    model_filepath = os.getenv('SAVE_TO', '/v3io/bigdata/netops/models')
    if not os.path.exists(model_filepath):
        os.makedirs(model_filepath)
    setattr(context, 'model_filepath', os.path.join(model_filepath, os.getenv('MODEL_FILENAME', 'netops.model')))

### Handler

In [60]:
def handler(context, event):
    
    # Get data
    df = get_data(context) 

    # Split to Train / Test datasets
    X_train, X_test, y_train, y_test = get_train_test_sets_from_data(context, df)
    print(X_train.columns)
    
    # Train
    model = xgb.XGBClassifier()
    model.fit(X_train, y_train)
    
    # Score
    score = model.score(X_test, y_test)
    
    # Save model
    pickle.dump(model, open(context.model_filepath + '.pickle', 'wb'))

## Test

In [58]:
# nuclio: ignore
init_context(context)

/conda/lib/python3.6/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


In [59]:
# nuclio: ignore
# init_context(context)
event = nuclio.Event(body='')
output = handler(context, event)
output

Index(['cpu_utilization_hourly', 'cpu_utilization_minute',
       'cpu_utilization_raw', 'latency_hourly', 'latency_minute',
       'latency_raw', 'packet_loss_hourly', 'packet_loss_minute',
       'packet_loss_raw', 'throughput_hourly', 'throughput_minute',
       'throughput_raw'],
      dtype='object')
['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11']
0.9974091063540604


## Deployment

In [ ]:
%nuclio deploy -p netops -n training -c

%nuclio: ['deploy', '-p', 'netops', '-n', 'training', '-c', '/User/netops/tutorials/demos/netops/Nuclio-Training.ipynb']
%nuclio: [nuclio.deploy] 2019-05-15 14:42:00,377 (info) Building processor image
%nuclio: [nuclio.deploy] 2019-05-15 14:43:11,142 (info) Pushing image
%nuclio: [nuclio.deploy] 2019-05-15 14:43:37,406 (info) Build complete
%nuclio: [nuclio.deploy] 2019-05-15 14:43:43,468 (info) Function deploy complete
%nuclio: [nuclio.deploy] 2019-05-15 14:43:43,474 done updating training, function address: 3.122.56.83:32342
%nuclio: function deployed
